In [19]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#get the directory of the csv file
train_data_dir = '/Users/serge/Documents/Github/Kaggle/kaggle_titanic/train.csv'
test_data_dir = '/Users/serge/Documents/Github/Kaggle/kaggle_titanic/test.csv'

#read csv to dataframe
df_train = pd.read_csv(train_data_dir)
df_test = pd.read_csv(test_data_dir)

In [20]:
#show the first 5 rows  
df_train.head()
df_test.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_encode,Age_bin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,4
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,7
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,5
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,6
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,6


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [21]:
#check nan value
df_train.isnull().sum()
df_test.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Sex_encode       0
Age_bin          0
dtype: int64

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [22]:
#group age by pclass
age_train = df_train.groupby('Pclass')['Age']
age_test = df_test.groupby('Pclass')['Age']
#show statistics summary 
age_train.describe()
age_test.describe()

,count,mean,std,min,25%,50%,75%,max
Pclass,,,,,,,,
1,216.0,38.062130,13.737970,0.92,29.00,37.0,47.25,80.0
2,184.0,29.825163,13.575362,0.67,23.00,29.0,36.00,70.0
3,491.0,24.824684,10.632997,0.42,20.25,24.0,29.00,74.0


,count,mean,std,min,25%,50%,75%,max
Pclass,,,,,,,,
1,98.0,40.918367,13.956799,6.00,30.0,42.0,50.75,76.0
2,88.0,28.777500,12.943458,0.92,21.0,26.5,35.25,63.0
3,146.0,24.027945,10.537105,0.17,18.5,24.0,29.75,60.5


In [23]:
#add the median value of age to nan values by each Pclass
df_train.loc[df_train['Age'].isnull(), 'Age'] = df_train.groupby('Pclass')['Age'].transform('median')
df_test.loc[df_test['Age'].isnull(),'Age'] = df_test.groupby('Pclass')['Age'].transform('median')
#check nan value
df_train.isnull().sum()
df_test.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Sex_encode       0
Age_bin          0
dtype: int64

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [7]:
from sklearn import preprocessing
#from sklearn.model_selection import train_test_split

In [24]:
#bring the label encoder
encoder = preprocessing.LabelEncoder()
#encode sex, female as 0 and male as 1
df_train['Sex_encode'] = encoder.fit_transform(df_train['Sex'])
df_test['Sex_encode'] = encoder.fit_transform(df_test['Sex'])

In [25]:
#create bins from 0 to 85 with step of 5 (the max age is 80)
bins = range(0, 85, 5)
#len(bins)=17
label_code = range(0,16)
df_train['Age_bin'] = pd.cut(df_train['Age'], bins, 
                                         labels=label_code) 
df_test['Age_bin'] = pd.cut(df_test['Age'], bins,
                                         labels=label_code)
#convert label to int 
df_train['Age_bin'] = df_train['Age_bin'].astype(int)
df_test['Age_bin'] = df_test['Age_bin'].astype(int)

In [10]:
#set features and target column
Features = np.array(['Pclass','SibSp','Parch','Sex_encode','Age_bin'])
Target = np.array(['Survived'])

In [26]:
#passing values to list
x = np.array(df_train[Features])
x_test = np.array(df_test[Features])
y = np.array(df_train[Target])
#convert to 1D array
y = y.ravel() 

In [12]:
#split data into train set(75%) and test set(25%) with shuffling of 42
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [27]:
#print('Training Features Shape:', x_train.shape)
#print('Training Target Shape:', y_train.shape)
#print('Testing Features Shape:', x_test.shape)
#print('Testing Target Shape:', y_test.shape)
print('Training Features Shape:', x.shape)
print('Training Target Shape:', y.shape)
print('Testing Features Shape:', x_test.shape)

Training Features Shape: (891, 5)
Training Target Shape: (891,)
Testing Features Shape: (418, 5)


In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [28]:
neigh = KNeighborsClassifier(n_neighbors=9)
neigh.fit(x, y)
predict = neigh.predict(x_test)
print(predict)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                     weights='uniform')

[0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 1 0 1 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 1 1 1 0 1 1
 1 1 0 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 1 0 1 1 1 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 1 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 0
 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0
 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 1 1 1 1 1 1 0 1 0 0 1]


In [31]:
df_test['Survived'] = pd.DataFrame(predict)

df_submission = df_test[{'PassengerId','Survived'}]

df_submission.to_csv (r'/Users/serge/Documents/Github/Kaggle/kaggle_titanic/submission.csv', index = False, header=True)

In [46]:
from sklearn.naive_bayes import GaussianNB

In [47]:
gnb = GaussianNB()
gnb.fit(x, y)
y_pred = gnb.predict(x_test)

GaussianNB(priors=None, var_smoothing=1e-09)

In [48]:
df_test['Survived_NB'] = pd.DataFrame(y_pred)

df_submission_NB = df_test[{'PassengerId','Survived_NB'}]


In [51]:
df_submission_NB.rename(columns={'Survived_NB': 'Survived'},inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [52]:
df_submission_NB.to_csv (r'/Users/serge/Documents/Github/Kaggle/kaggle_titanic/submission_NB1.csv', index = False, header=True)

In [50]:
df_submission_NB

,PassengerId,Survived_NB
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
